<a href="https://colab.research.google.com/github/slxuphys/physics-demo/blob/main/billard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code Preparation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import random

## Pool Table

### functions & class

In [ ]:
class Ball(object):
    '''a ball
    '''
    def __init__(self,r,v,m,radius,color): #initialize the object, including the radius
        self.r=np.array(r)
        self.v=np.array(v)
        self.m=m
        self.radius = radius
        self.path={'x':[r [0]],'y':[r[1]]} #self.path is a dictionry, containing two keys 'x', 'y', the value of each key is a list
        self.graph=plt.Circle((self.path['x'][0], self.path['y'][0]),radius=self.radius,zorder=2, color=color)
        self.line=None
        self.color=color
        #r,v,m,path are attributes of ball
    def record(self):
        self.path['x'].append(self.r[0]) #append the current x position to path
        self.path['y'].append(self.r[1]) #append the current y position to path
    def draw(self,ax):
        ax.add_patch(self.graph)
        line = ax.plot(self.path['x'],self.path['y'], color=self.color)
        self.line = line[0]
    def update_position(self,indx):
        self.graph.center=self.path['x'][indx], self.path['y'][indx]
        self.line.set_data(self.path['x'][indx-500:indx],self.path['y'][indx-500:indx])
    def undraw(self):
        self.graph.remove()


def trajectory(frame_indx,ball,factor):
    indx = factor*frame_indx
    for i_ball in range(len(ball)):
        ball[i_ball].update_position(indx)

In [ ]:
def motion_billiard(dt, num_step, ball, length, width, t=0):
    def boundary(ball):
        '''checking if the ball is within the boundary
        '''
        if ball.r[0]-ball.radius<0 or ball.r[0] + ball.radius>length:
            ball.v[0]=-ball.v[0]
        if ball.r[1]-ball.radius<0 or ball.r[1] + ball.radius>width:
            ball.v[1]=-ball.v[1]
        return ball
    time_list=[t]
    num_ball = len(ball)
    #starting iteration
    for i_step in range(num_step):
        ##update position
        t=t+dt
        time_list.append(t)
        for i_ball in range(num_ball):
            ball[i_ball].r += ball[i_ball].v*dt
            ball[i_ball] = boundary(ball[i_ball])
            ball[i_ball].record()
    #Visualization
    fig, ax = plt.subplots(figsize=(4,4*width/length))
    fig.set_facecolor('grey')
    ax.plot([0,0],[0,width],'w')
    ax.plot([length,length],[0,width],'w')
    ax.plot([0,length],[0,0],'w')
    ax.plot([0,length],[width,width],'w')
    ax.axis('equal')
    ax.axis('off');
    for i_ball in range(num_ball):
        ball[i_ball].draw(ax)
    plt.close()
    num_frame = int(len(ball[0].path['x'])/100) # the total number of frames
    anim = animation.FuncAnimation(fig, lambda x: trajectory(x,ball,100), frames=num_frame, interval=40)
    return anim


### run

In [ ]:
ball1 = Ball(r=np.array([7.0,5.0]), v=np.array([-1,4.0]), radius=0.1, m=1.0, color='tab:blue')
ball2 = Ball(r=np.array([7.2,5.2]), v=np.array([-1,4.0]), radius=0.1, m=4.0, color='tab:red')
anim = motion_billiard(dt=0.0004, num_step=100000, ball=[ball1,ball2], length=14, width=10)
HTML(anim.to_html5_video())

In [ ]:
writer = animation.PillowWriter(fps=15,
                                metadata=dict(artist='Me'),
                                bitrate=1800)
anim.save('scatter.gif', writer=writer)

In [ ]:
def motion_billiard_chaos(dt, num_step, ball, length, width, t=0):
    def boundary(ball):
        '''checking if the ball is within the boundary
        '''
        r1 = np.array((0, width/2))
        r2 = np.array((length, width/2))
        if ball.r[1]-ball.radius<0 or ball.r[1] + ball.radius>width:
            ball.v[1]=-ball.v[1]
        if np.linalg.norm(ball.r-r1)>width/2 and ball.r[0]<0:
            n = (ball.r - r1)
            n = n/np.linalg.norm(n)
            v_n = np.dot(ball.v,n)*n
            v_t = ball.v - v_n
            ball.v = -v_n + v_t
        if np.linalg.norm(ball.r-r2)>width/2 and ball.r[0]>length:
            n = (ball.r - r2)
            n = n/np.linalg.norm(n)
            v_n = np.dot(ball.v,n)*n
            v_t = ball.v - v_n
            ball.v = -v_n + v_t

        return ball
    time_list=[t]
    num_ball = len(ball)
    #starting iteration
    for i_step in range(num_step):

       ##update position
        t=t+dt
        time_list.append(t)
        for i_ball in range(num_ball):
            ball[i_ball].r += ball[i_ball].v*dt
            ball[i_ball] = boundary(ball[i_ball])
            ball[i_ball].record()
    #Visualization
    fig, ax = plt.subplots(figsize=(4,4*width/length))
    y = np.linspace(0,width,100)
    x = np.sqrt(width**2/4-(y-width/2)**2)
    ax.plot(-x,y,'w')
    ax.plot(x+length,y,'w')

    ax.plot([0,length],[0,0],'w')
    ax.plot([0,length],[width,width],'w')
    ax.axis('equal')
    ax.axis('off');
    fig.set_facecolor('darkgrey')
    for i_ball in range(num_ball):
        ball[i_ball].draw(ax)
    plt.close()
    num_frame = int(len(ball[0].path['x'])/100) # the total number of frames
    anim = animation.FuncAnimation(fig, lambda x: trajectory(x,ball,100), frames=num_frame, interval=40)
    return anim


In [ ]:
ball1 = Ball(r=np.array([7.0,5.0]), v=np.array([-1,4.0]), radius=0.1, m=1.0, color='tab:blue')
ball2 = Ball(r=np.array([7.2,5.2]), v=np.array([-1,4.0]), radius=0.1, m=4.0, color='tab:red')
anim = motion_billiard_chaos(dt=0.0004, num_step=100000, ball=[ball1,ball2], length=14, width=10)
HTML(anim.to_html5_video())

In [ ]:
writer = animation.PillowWriter(fps=15,
                                metadata=dict(artist='Me'),
                                bitrate=1800)
anim.save('chaos.gif', writer=writer)